# Visualization notebook

This notebook is meant for visualizing stuff and testing code. 

In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pandas as pd
from pathlib import Path
from more_itertools import chunked
from dataclasses import dataclass
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
from gram2vec.featurizers import GrammarVectorizer, make_document
warnings.filterwarnings("ignore", category=FutureWarning)

# PAN 2022 Summary Stats & Visualizations


In [2]:
@dataclass
class Author:
    """
    Stores author information in an easy to work with format
    
    :param author_id: unique author id
    :param fixed_texts: list of author documents with regex fixes
    :param raw_texts: list of author documents without regex fixes
    :param discourse_types: list of discourse types
    
    Note: fixed_docs, raw_docs, and discourse_types are all 1 - 1 corresponding
    """
    author_id:str
    fixed_texts:list[str]
    raw_texts:list[str]
    discourse_types:list[str]
    
    def get_token_counts(self) -> list[int]:
        return [len(word_tokenize(author_doc)) for author_doc in self.fixed_texts]
    
    def get_total_docs(self) -> int:
        return len(self.fixed_texts)
    
    def count_dicourse_type(self, dtype:str) -> int:
        return Counter(self.discourse_types)[dtype]
        
def load_json(path:str) -> dict[str, list[dict]]:
    with open(path, "r") as fin:
        data = json.load(fin)
        return data

def extract_from_dict(author_entry:dict, to_extract:str) -> list[str]:
    return [entry[to_extract] for entry in author_entry]
    
def create_author_list(preprocessed_data:dict[str, list[dict]]) -> list[Author]:
    """
    Converts the preprocessed data into a list of Author objects
    """
    authors = []
    for author_id in preprocessed_data.keys():
        author_entry = preprocessed_data[author_id]
        fixed_texts = extract_from_dict(author_entry,"fixed_text")
        raw_texts = extract_from_dict(author_entry,"raw_text")
        discourse_types = extract_from_dict(author_entry,"discourse_type")
            
        authors.append(Author(author_id, fixed_texts, raw_texts, discourse_types))
        
    return authors

def get_doc_token_stats(authors:list[Author]) -> tuple[float, float]:
    """Gets the mean and std of tokens per document"""
    all_doc_token_counts = []
    for author in authors:
        all_doc_token_counts.extend(author.get_token_counts())
    return np.mean(all_doc_token_counts), np.std(all_doc_token_counts)
    
def make_author_df(authors:list[Author]) -> pd.DataFrame:
    
    author_maps = defaultdict(list)
    for author in authors:
        author_maps["author_id"].append(author.author_id)
        author_maps["total_token_count"].append(sum(author.get_token_counts()))
        author_maps["Total docs"].append(author.get_total_docs())
        author_maps["Emails"].append(author.count_dicourse_type("email"))
        author_maps["Memos"].append(author.count_dicourse_type("memo"))
        author_maps["Txt msgs"].append(author.count_dicourse_type("text_message"))
        author_maps["Essays"].append(author.count_dicourse_type("essay"))
        
    return pd.DataFrame(author_maps)

data = load_json("data/pan22/preprocessed/author_doc_mappings.json")
all_authors = create_author_list(data)
df = make_author_df(all_authors)




## Feature testing ground

In [9]:
g2v = GrammarVectorizer()

def get_all_documents(data_path:str, text_type="fixed_text") -> list[str]:
    """Aggregates all documents into one list"""
    all_documents = []
    for author_entries in load_json(data_path).values():
        for entry in author_entries:
            all_documents.append(make_document(entry[text_type], g2v.nlp))
            
    return all_documents

all_documents = get_all_documents("eval/pan22_splits/knn/train.json")

array([8.48287113e-02, 7.99347471e-02, 7.17781403e-02, 6.19902121e-02,
       2.61011419e-02, 6.68841762e-02, 8.15660685e-03, 1.66394780e-01,
       4.89396411e-03, 2.44698206e-02, 1.33768352e-01, 6.52528548e-03,
       8.80913540e-02, 3.09951060e-02, 0.00000000e+00, 1.37030995e-01,
       3.26264274e-03, 4.89396411e-03, 3.87596899e-02, 3.25581395e-02,
       3.87596899e-02, 3.87596899e-02, 2.17054264e-02, 4.80620155e-02,
       2.48062016e-02, 2.48062016e-02, 3.25581395e-02, 1.70542636e-02,
       2.48062016e-02, 2.32558140e-02, 2.94573643e-02, 2.01550388e-02,
       1.39534884e-02, 2.32558140e-02, 1.08527132e-02, 1.24031008e-02,
       1.70542636e-02, 4.65116279e-03, 2.01550388e-02, 1.55038760e-02,
       6.20155039e-03, 1.08527132e-02, 4.65116279e-03, 6.20155039e-03,
       1.24031008e-02, 1.24031008e-02, 9.30232558e-03, 1.24031008e-02,
       1.70542636e-02, 1.39534884e-02, 3.10077519e-03, 1.08527132e-02,
       7.75193798e-03, 0.00000000e+00, 1.08527132e-02, 7.75193798e-03,
      

In [6]:
from more_itertools import flatten

def get_morph_tag_distr(documents:list, morph_tag:str) -> dict:
    
    morph_tag_map = defaultdict(int)
    for document in documents:
        doc_morphs = list(flatten([token.morph.get(morph_tag) for token in document.spacy_doc]))
        for morph in doc_morphs:
            morph_tag_map[morph] += 1
    return morph_tag_map

get_morph_tag_distr(all_documents, "Mood")

    

defaultdict(int, {'Ind': 7327})

## Visuals

In [ ]:
from data.scripts.pan_create_bins import get_train_authors_sorted_by_docfreq

def bin_authors(iterable) -> tuple[list[str], ...]:
    return tuple(chunked(iterable, 7)) 

def make_doc_avg_labels(sorted_dict):
       
       labels = []
       for bin in bin_authors(list(sorted_dict.values())):
              labels.append(round(np.mean(bin), 2))
       return labels

train_path = "eval/pan22_splits/knn/train.json"
train_authors_sorted = get_train_authors_sorted_by_docfreq(train_path)
labels = make_doc_avg_labels(train_authors_sorted)

# k = 6

r_at_1 = np.array([0.02857142857,0.2285714286,0.1428571429,0.1428571429,0.2285714286,0.1142857143,0.2285714286,0.5142857143])

r_at_8 = np.array([0.2571428571,0.5142857143,0.5142857143,0.5142857143,0.7142857143,0.6857142857,0.7142857143,0.7428571429,])


df = pd.DataFrame({
       "Bin labels": labels,
       "R@1": r_at_1,
       "R@8": r_at_8
})

sns.set_style("darkgrid")


sns.lineplot(data=df, x="Bin labels", y="R@1",color="blue",marker="o", label="R@1")
sns.lineplot(data=df, x="Bin labels", y="R@8",color="green",marker="o", label="R@8")
plt.xlabel("Avg document count")
plt.ylabel("Score")
plt.title("Binned author scores")
plt.legend()

plt.show()

In [ ]:

sns.set_style("darkgrid")
df["Total docs"].hist(bins=7)
plt.title("Document counts per author")
plt.xlabel("# of documents")
plt.ylabel("# of authors")
plt.show()

The following cell contains deprecated information

In [ ]:
ALL_FEATS_ACCS = [0.0, 0.05714285714285714, 0.11428571428571428, 0.17142857142857143, 0.2, 0.2, 0.2857142857142857, 0.5428571428571428]
HALF_FEATS_ACCS = [0.0, 0.02857142857142857, 0.08571428571428572, 0.11428571428571428, 0.22857142857142856, 0.22857142857142856, 0.22857142857142856, 0.4]

old_df = pd.DataFrame(
    {"Full features": ALL_FEATS_ACCS,
     "Half features": HALF_FEATS_ACCS,
     "Bin labels":labels}
)


sns.lineplot(data=old_df, x="Bin labels", y="Full features",color="blue", label="Full features")
sns.lineplot(data=old_df, x="Bin labels", y="Half features", color="red", label="Half features")
plt.xlabel("Avg document count")
plt.ylabel("R@1 score")
plt.title("R@1 Development bin scores")
plt.legend()

plt.show()

# Discourse related stuff

In [ ]:
def load_all_discourse_types(path:str) -> tuple[list,list,list]:
    """Loads the preprocessed data and sorts it by discourse type"""
    preprocessed = load_json(path)
    author_ids = preprocessed.keys()
    emails = []
    memos = []
    txt_msgs = []
    essays = []
    for author_id in author_ids:
        for author_entry in preprocessed[author_id]:
            dtype = author_entry["discourse_type"]
            fixed = author_entry["fixed_text"].split()
            
            if  dtype == "email":
                emails.append(fixed)
                
            if  dtype == "memo":
                memos.append(fixed)
                
            if  dtype == "text_message":
                txt_msgs.append(fixed)
                
            if  dtype == "essay":
                essays.append(fixed)
    return emails, memos, txt_msgs, essays
  
def get_avg_tokens(dtype:list[list[str]]) -> int:
    
    token_counts = []
    for tokens in dtype:
        token_counts.append(len(tokens))
    return np.mean(token_counts)
              
            
emails, memos, txt_msgs, essays = load_all_discourse_types("data/pan22/preprocessed/preprocessed_data.json")


print(get_avg_tokens(emails))
print(get_avg_tokens(memos))
print(get_avg_tokens(txt_msgs))
print(get_avg_tokens(essays))
    

In [ ]:
sns.set_style("darkgrid")

dtype_df = df[["Emails", "Txt msgs", "Essays", "Memos"]].sum()
dtype_df.plot.bar(color=["teal", "lightpink", "orange", "brown"])
plt.title("Discourse type counts")
plt.ylabel("Count")
plt.xticks(rotation=45)

plt.show()